### Directly Download from GC and save as netcdf files
- This is for those who cannot use zarr/python for processing the CMIP6 datasets
- Please note that the netcdf files have CF-compliant time grids, but might not be what you are used to

In [34]:
import numpy as np
import pandas as pd
import os
import gcsfs #google cloud file system. 
import xarray as xr
import warnings
from glob import glob # use * !
import scipy.io as sio

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (8,5)

In [35]:
from utls import search_df, add_time_info, get_zdict #extra functions

### Initialization

In [36]:
def compute_area_and_seasonal_mean(danom, xlim, ylim, slim, mask=1):
    """
    Weights each grid point by the cos(latitude), computes area mean, normalizing by areaa mean of the weights
    returns:
        DataArray:  global mean for each model
    """  
    xlim = np.array(xlim)
    xlim += (xlim<0 )*360
    if xlim[0]>xlim[1]:
        lon_sel = (danom.lon>xlim[0])+(danom.lon<xlim[1])
    else:
        lon_sel = (danom.lon>xlim[0])*danom.lon<xlim[1]
    
    if type(mask)!=int:
        mask = (xr.ones_like(danom)*mask).isel({'lat': (danom.lat>ylim[0])*(danom.lat<ylim[1]), 'lon': lon_sel})
    
    danom = danom.isel({'lat': (danom.lat>ylim[0])*(danom.lat<ylim[1]), 
                        'lon': lon_sel, 
                        'time': (danom['time'].dt.month >= slim[0])*(danom['time'].dt.month <= slim[1]),
                       })
    coslat = np.cos(np.deg2rad(danom.lat))
    weights = xr.ones_like(danom)*coslat*mask
    weight_mean = weights.mean(['lat','lon'], keep_attrs=True)
    area_mean = (danom * weights).mean(['lat','lon'], keep_attrs=True)/weight_mean
    if area_mean.time.dtype!='datetime64[ns]':
        area_mean['time'] = area_mean.indexes['time'].year
        return area_mean.groupby('time').mean(dim='time', keep_attrs=True)
        #area_mean.indexes['time'].to_datetimeindex()
        #there are lots of dftime.DatetimeNoLeap. this would matter a touch if I used a weighted average.
    else:   
        #month_length = danom.time.dt.days_in_month
        #this actually isn't right! Let's just do an unweighted mean for now :(
        #weights = month_length/sum(month_length[slim[0]:slim[1]+1])
        #Sm = (weights*area_mean)
        #return Sm.groupby(grp).sum(dim='time', keep_attrs=True)
        return area_mean.groupby(area_mean.time.dt.year).mean(dim='time', keep_attrs=True)
    
    #technically this is inconsistent handling; one makes a dimension called "time" and the other makes "years". I have to deal with that in my matlab code
    
plev19 = [100000, 92500, 85000, 70000, 60000, 50000, 40000, 30000, 25000, 20000, 15000, 10000, 7000, 5000, 3000, 2000, 1000, 500, 100]
plev19_bnds = [100000,95000,90000,75000,65000,55000,45000,35000,27000,23000,17000,12500,8500,6500,4000,2500,1500,700,300,100]

In [37]:
#danom = ds0

#xlim = [-20,40]
#ylim = [12,18]
#slim = [7,9]
def calc_stp(dl):
    l = dl.values
    stp = max(l[1:] - l[0:-1])
    return stp

def compute_conv(danom, xlim, ylim, slim, plim):
    """
    Calculates the mean flow convergence of moisture on monthly scale into the Sahel region.
    returns convergence at each pressure level. add pressure level ranges to integrate over.
    """  
    xlim = np.array(xlim)
    xlim += (xlim<0)*360
    lt_stp = calc_stp(danom.lat)
    ln_stp = calc_stp(danom.lon)
    if xlim[0]>xlim[1]:
        lon_sel_2 = (danom.lon>(xlim[0]-ln_stp))+(danom.lon<(xlim[1]+ln_stp))
    else:
        lon_sel_2 = (danom.lon>(xlim[0]-ln_stp))*(danom.lon<(xlim+ln_stp))

    danom = danom.isel({'time': (danom['time'].dt.month >= slim[0])*(danom['time'].dt.month <= slim[1]),
                         'lat': (danom.lat>(ylim[0]-lt_stp))*(danom.lat<(ylim[1]+lt_stp)),
                         'lon': lon_sel_2,
                         'plev': danom.plev==plim #when I can figure out the bounds change this to an integral
                       })
    danom['qfu'] = danom['hus']*danom['ua']
    danom['qfv'] = danom['hus']*danom['va']
    if xlim[0]>xlim[1]:
        lon_sel = (danom.lon>xlim[0])+(danom.lon<xlim[1])
    else:
        lon_sel = (danom.lon>xlim[0])*danom.lon<xlim[1]
    lat_sel = (danom.lat>ylim[0])*(danom.lat<ylim[1])
    danom.load()
    dS = danom['qfv'].isel({'lat': danom.lat<(ylim[0]+lt_stp), 'lon': lon_sel}).mean(['lat'], keep_attrs=True)
    dN = -danom['qfv'].isel({'lat': danom.lat>(ylim[1]-lt_stp), 'lon': lon_sel}).mean(['lat'], keep_attrs=True)
    dW = danom['qfu'].isel({'lat': lat_sel, 'lon': danom.lon<(xlim[0]+ln_stp)}).mean(['lon'], keep_attrs=True)
    dE = -danom['qfu'].isel({'lat': lat_sel, 'lon': danom.lon>(xlim[1]-ln_stp)}).mean(['lon'], keep_attrs=True)
    coslat = np.cos(np.deg2rad(ylim))
    d_conv = (coslat[0]*dS + coslat[1]*dN).sum(['lon'], keep_attrs=True)+(dW+dE).sum(['lat'], keep_attrs=True)
    if d_conv.time.dtype!='datetime64[ns]':
        d_conv['time'] = d_conv.indexes['time'].year
        return d_conv.groupby('time').mean(dim='time', keep_attrs=True)
    else:   
        return d_conv.groupby(d_conv.time.dt.year).mean(dim='time', keep_attrs=True)

In [38]:
# Where to write local netcdf files:
username = os.environ['JUPYTERHUB_USER']
mach = os.uname()[1]

zarr_local = f'/home/{username}/netcdf/cmip6/preprocessed'
if not os.path.exists(zarr_local):
    print(f'Please create the directory {zarr_local}')
    

In [39]:
# This is the master CMIP6 Google Cloud catalog
df_cloud = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv', dtype='unicode')
df_cloud.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version,status,severity,issue_url
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,hus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706,good,none,none
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,tauu,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706,resolved,high,https://errata.es-doc.org/static/view.html?uid...
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,tasmin,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706,good,none,none
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,tasmax,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706,good,none,none
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,hurs,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706,good,none,none


### Choose basic configuration parameters

In [21]:
# Here we search the CMIP6 data for the datasets you need - using the same keywords as at the ESGF sites
#       https://esgf-node.llnl.gov/search/cmip6/

debug = False

# must choose ONE table_id  (only works for *mon or *day)
table_id = 'Amon'

#must choose LIST of experiments, variables
experiments = ['historical']#,'hist-aer', , 'hist-nat', 'hist-GHG', 'piControl' 'amip-hist',
variables = ['ts']

location = 'Ocean'#'Sahel'#

# can specify 'All' or give a list or string
#sources = ['MPI-ESM1-2-LR']#GFDL-CM4']#'ACCESS-ESM1-5']#'CNRM-ESM2-1']#'NorESM2-LM']# 'GFDL-ESM4'['CanESM5-CanOE']#CMCC-CM2-SR5']#'CIESM']#MCM-UA-1-0']  #AWI-CM-1-1-MR']#SAM0-UNICON']# omit the [] to get all models with CESM2 in their name
sources = 'All'
#members = ['r6i1p1f1']#r1i1p1f1']#'r9i1p1f2']#
members = 'All'

In [22]:
search = {'table_id':table_id}
search['experiment_id'] = experiments
search['variable_id'] = variables
if sources != 'All':
    search['source_id'] = sources
if members != 'All':
    search['member_id'] = members
    
df_available = search_df(df_cloud, **search)

print('number of matching datasets',len(df_available))

#523 historical simulations < 536 on the cite directly. Do I want to figure out which simulations are missing? 
# Or do I trust that they are missing for a reason?

number of matching datasets 585


In [23]:
df_available.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version,status,severity,issue_url
1615,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r3i1p1f1,Amon,zg,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/hist...,NaN,20180701,good,none,none
2589,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r2i1p1f1,Amon,zg,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/hist...,NaN,20180701,good,none,none
19295,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Amon,zg,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180701,wontfix,medium,https://errata.es-doc.org/static/view.html?uid...
22313,CMIP,IPSL,IPSL-CM6A-LR,historical,r5i1p1f1,Amon,zg,gr,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803,good,none,none
22319,CMIP,IPSL,IPSL-CM6A-LR,historical,r4i1p1f1,Amon,zg,gr,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803,good,none,none


In [24]:
# For proper debugging, it is helpful to add time grid information to dataframe:
if debug:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        dfa = add_time_info(df_available)
else:
    dfa = df_available.copy()

In [25]:
#only use the MASK code for TS!
if location=='Ocean':
    search_mask = {'table_id':'fx'}
    search_mask['experiment_id'] = ['historical', 'piControl', '1pctCO2','hist-resIPO','hist-1950HC']
    search_mask['variable_id'] = ['sftlf']
    if sources != 'All':
        search_mask['source_id'] = sources
    if members != 'All':
        search_mask['member_id'] = members
    historical_mask = search_df(df_cloud, **search_mask)

    print('number of mask datasets',len(historical_mask))

In [26]:
#only use MASK code for TS!
if location=='Ocean':
    def get_ids(dfa, id_name):
        zdicts = list(map(get_zdict, list(dfa.zstore.values)))
        return set(map(lambda x: x[id_name], zdicts))

    id_name = 'source_id'
    historical_mask_models = get_ids(historical_mask, id_name)
    sst_models = get_ids(dfa, id_name)

    maskable_models = sst_models.intersection(historical_mask_models)

    gsurls = np.array([gsurl for gsurl in dfa.zstore.values if get_zdict(gsurl)['source_id'] in maskable_models])
    missing = np.array([gsurl for gsurl in dfa.zstore.values if not get_zdict(gsurl)['source_id'] in maskable_models])

    masks = historical_mask.groupby('source_id').first()

    gsurls


In [27]:
if location=='Ocean':
    missing

In [28]:
# for pr:
if location=='Sahel':
    gsurls = np.array(dfa.zstore.values)

In [29]:
if debug:
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)

    dm = dfa[['experiment_id','source_id','member_id','variable_id','start','stop']].groupby([
             'experiment_id','start','stop','source_id']).nunique()[['member_id']]

    table = pd.DataFrame.pivot_table(dm,
                                     values='member_id',
                                     index=['source_id','start','stop'],
                                     columns=['experiment_id'],
                                     aggfunc=np.sum,
                                     fill_value=0)
    print(table)

In [30]:
dfa.issue_url.unique()

array(['none',
       'https://errata.es-doc.org/static/view.html?uid=2f6b5963-f87e-b2df-a5b0-2f12b6b68d32',
       'https://errata.es-doc.org/static/view.html?uid=5ebabff0-388f-07bc-b2cf-d44dcbb2940f',
       'https://errata.es-doc.org/static/view.html?uid=ad9e4213-f254-e930-a143-030a4b0f8a32',
       'https://errata.es-doc.org/static/view.html?uid=e6b642cf-d449-dfa6-a9da-1d1d1e1c0ecd'],
      dtype=object)

In [31]:
fs = gcsfs.GCSFileSystem(token='anon', access='read_only') #the actual files, not the list of files woohoo FILE SYSTEM
#fs.get_mapper()

In [33]:
#gsurls = dfa.zstore.values #zstore is the url where the data is stored

conv = False
replace = False
start_month = 5 #7
end_month = 7 #9

ds_list = []
ds_failed_list = []

def update_vars(ds):
    for var in [var for var in ds.coords]:
        if 'bounds' in var:
            nvar = var.replace('bounds','bnds')
            #print(var,nvar)
            ds = ds.rename({var:nvar})
        if 'latitude' in var:
            nvar = var.replace('latitude','lat')
            #print(var,nvar)
            ds = ds.rename({var:nvar})
        if 'longitude' in var:
            nvar = var.replace('longitude','lon')
            #print(var,nvar)
            ds = ds.rename({var:nvar})
    return ds

if conv:
    search_uv = search.copy()
    search_uv['source_id'] = [model]
    search_uv['member_id'] = [run]
    search_u = search_uv.copy()
    search_v = search_uv.copy()
    search_u['variable_id'] = ['ua']
    search_v['variable_id'] = ['va']
    
for gsurl in gsurls:
    zdict = get_zdict(gsurl) #naomi func for metadata
    institution = zdict['institution_id']
    model = zdict['source_id']
    run = zdict['member_id']
    variable = zdict['variable_id']
    expt = zdict['experiment_id']
    if conv:
        filename = f'{variable}_conv_925_{institution}_{model}_{run}'
    else:
        filename = f'{variable}_{institution}_{model}_{run}'
    ncdir = f'{zarr_local}/{expt}/{start_month}-{end_month}'
    ncfile = f'{ncdir}/{filename}.nc'
    
    if(not replace):
        ncfiles = glob(ncfile) #check not to double-download files
        if len(ncfiles) > 0:
            print(ncfiles, 'already exists')
            continue
            
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        ds0 = xr.open_zarr(fs.get_mapper(gsurl),consolidated=True) #gets info about the file. get_mapper! always use consolidated=True
        if variable=='ts':
            mask_ds = xr.open_zarr(fs.get_mapper(masks['zstore'][model]),consolidated=True) 
        #if variable=='ta' or variable=='zg':
         #   ds0 = ds0.isel({'plev': ds0.plev==100})
        if conv:
            U = np.array(search_df(df_cloud, **search_u).zstore.values)
            V = np.array(search_df(df_cloud, **search_v).zstore.values)
            if len(U) == 1 and len(V) == 1:
                dsU = xr.open_zarr(fs.get_mapper(U[0]),consolidated=True)
                dsV = xr.open_zarr(fs.get_mapper(V[0]),consolidated=True)
                ds0['va'] = dsV['va']
                ds0['ua'] = dsU['ua']
            else:
                continue
        
    #month_length = ds.time.dt.days_in_month #for some reason the first one doesn't have this...
        
    ds0 = update_vars(ds0)
    
    if location=='Ocean':
        mask_ds = update_vars(mask_ds)
        ls_mask = np.floor(1-mask_ds.sftlf.values/100)
    
    try:
        #if variable=='hus':
        #    ds = ds0.isel({'plev':ds0.plev==8.5e4})
        #else:
        ds = ds0
        if location=='Ocean':
            #TODO don't assume it's ts. make NARI and p1 only for ts.
            NA = compute_area_and_seasonal_mean(ds, [-75,-15], [10,40], [start_month, end_month])
            GT = compute_area_and_seasonal_mean(ds, [0,360],[-20,20],[start_month, end_month], mask=ls_mask)
            SA = compute_area_and_seasonal_mean(ds, [-30, 20], [-20,10], [start_month, end_month], mask=ls_mask)
            md = compute_area_and_seasonal_mean(ds, [-6,36], [30,40], [start_month, end_month], mask=ls_mask)
            TA = compute_area_and_seasonal_mean(ds, [-75,-15], [5,15], [start_month, end_month])
            Sm = NA
            Sm = Sm.rename_vars({'ts':'NA'})
            Sm['GT'] = GT.ts
            Sm['NARI'] = NA.ts - GT.ts
            Sm['p1'] = NA.ts + GT.ts
            Sm['SA'] = SA.ts
            Sm['md'] = md.ts
            Sm['TA'] = TA.ts
        elif conv:
            Sm = compute_conv(ds, [-20,40], [12,18], [start_month, end_month], 92500)
        else:
            Sm = compute_area_and_seasonal_mean(ds, [-20,40], [12,18], [7,9])
  
    except ValueError:
        print(f'value error for {ncfile}')
        ds_failed_list += [ds]
        continue
    #have to customize this to the variable I'm using!
    if variable=='ts' or variable=='tas' or variable=='ta':
        if not ds[variable].attrs['units']=='K':
            print("cannot comprehend units ({}), skipping model {}".format(ds[variable].attrs['units'], model))
            continue
    elif variable=='pr':
        if ds.pr.attrs['units'] == 'kg m-2 s-1':
            Sm *= 86400
        else:
            print("cannot comprehend units ({}), skipping model {}".format(ds.pr.attrs['units'], model))
            continue
    elif variable=='zg':
        if not ds[variable].attrs['units']=='m':
            print("cannot comprehend units ({}), skipping model {}".format(ds[variable].attrs['units'], model))
            continue
    elif (variable=='huss'):
        Sm *= float(ds.huss.attrs['units'])*1000 #convert to g/kg
    elif (variable=='hus'):
        Sm *= float(ds.hus.attrs['units'])*1000 #convert to g/kg
    else:
        print("need to make new units case for variable {} with units {}".format(variable, ds[variable].attrs['units']))

    os.system(f'mkdir -p {ncdir}')
    try:
        Sm.to_netcdf(ncfile,mode='w',unlimited_dims=['time','year'])  #saves the file. Don't have to do this before I'm ready! But ds is replaced each time...
        ds_list += [Sm]
        print(f'saving file {ncfile}')
    except ValueError:
        print(f'value error for {ncfile}')
        ds_failed_list += [Sm]
        continue    
        
    #ok I got an error for a model which uses i/j coordinates instead of lat lon! OY VEY...
    

['/home/rebecca/netcdf/cmip6/preprocessed/historical/zg_NOAA-GFDL_GFDL-ESM4_r3i1p1f1.nc'] already exists
['/home/rebecca/netcdf/cmip6/preprocessed/historical/zg_NOAA-GFDL_GFDL-ESM4_r2i1p1f1.nc'] already exists
['/home/rebecca/netcdf/cmip6/preprocessed/historical/zg_NOAA-GFDL_GFDL-CM4_r1i1p1f1.nc'] already exists
['/home/rebecca/netcdf/cmip6/preprocessed/historical/zg_IPSL_IPSL-CM6A-LR_r5i1p1f1.nc'] already exists
['/home/rebecca/netcdf/cmip6/preprocessed/historical/zg_IPSL_IPSL-CM6A-LR_r4i1p1f1.nc'] already exists
['/home/rebecca/netcdf/cmip6/preprocessed/historical/zg_IPSL_IPSL-CM6A-LR_r9i1p1f1.nc'] already exists
['/home/rebecca/netcdf/cmip6/preprocessed/historical/zg_IPSL_IPSL-CM6A-LR_r10i1p1f1.nc'] already exists
['/home/rebecca/netcdf/cmip6/preprocessed/historical/zg_IPSL_IPSL-CM6A-LR_r13i1p1f1.nc'] already exists
['/home/rebecca/netcdf/cmip6/preprocessed/historical/zg_IPSL_IPSL-CM6A-LR_r14i1p1f1.nc'] already exists
['/home/rebecca/netcdf/cmip6/preprocessed/historical/zg_IPSL_IPSL

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (8,5)

ds.ts[0].plot()
(ds*xr.ones_like(ds)*np.cos(np.deg2rad(ds.lat))*ls_mask).ts[0].plot()

In [ ]:
Sm

In [ ]:
! tree -L 9 ~/CMIP6-downloads #unix tree of created files; I didn't create any.

In [ ]:
! du -sh ~/CMIP6-downloads/*/*/*/*

In [ ]:
ds = xr.open_dataset('/home/naomi/CMIP6-downloads/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/sfcWind/gn/sfcWind.nc')

In [ ]:
ds.sfcWind.plot(vmin=0,vmax=12)

In [ ]:
#Convert longitude coordinates from 0-359 to -180-179:

ds2 = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')

#or

ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
ds = ds.sortby(ds.lon)

In [ ]:
ds2.sfcWind[0].plot()